# II. Newton, Rhapson and a Calculator: Iterative Algorithms &mdash; Part I

## Last Week in Babylon

To begin this week's class, we will quickly revisit what we did last time. First we shall import the `math` module &mdash; always a good idea!

Recall the previous lesson: we explored some of the basic tools to perform calculations in Python 3, and put them together to implement the Babylonian square root algorithm. The algorithm is a simple iterative process to estimate $r$, the square root of a given number $s=r^2$:

- Take an initial guess for $r$, call it $x_0$. 
- Compute the update $x_{n+1}$ from the previous step $x_n$ using the rule:
$$x_{n+1}=\frac{x_n+\frac{s}{x_n}}{2}.$$
- Repeat until the guess is no longer changing.

We tried a few different implementations last time, and eventually settled for:

In [ ]:
def babylonianRoot(s):
    x=s
    
    for k in range(0,10000):
        xNext=(x+s/x)/2
        difference = abs(xNext-x)
        print(f"Difference: {difference}")
        if difference==0:
            break
                    
        x=xNext
    
    return x

You might rememeber that the algorithm showed excellent convergence. For instance, $\sqrt{\pi}$ is computed in only 6 iterations:

In writing the algorithm in Python we made use of several tools. We learnt how to define our own *functions*:


In [ ]:
def myFunction(x):
    return 2*x

myFunction(3)

In [ ]:
def myOtherFunction():
    return 17

myOtherFunction()

In [ ]:
def yetAnotherFunction():
    print("Hello")

yetAnotherFunction()

In [ ]:
def printMyNumber(x):
    print("My number is x")
    print(f"My number is {x}, and its half is {x/2}, and its square is {x*x}")
    print("My number is "+str(x))
    
printMyNumber(15)

We also saw how to use *`for` loops* to implement iteration or repetition:

In [ ]:
for k in [1,2,5,14]:
    print(k)

Last, but not least, we began to use *`if` statements* in order to execute certain parts of the code conditionally:

In [ ]:
x=-1

if x<0:
    print("Negative")
else:
    print("Positive")

### Solving a Fixed Point Equation

Last week we set the task of solving an equation:
$$\cos(x)=x.$$
This, which might seem like a completely new problem, is actually very similar to the Babylonian square root; a solution can be found by iteration!

![calculator][logo]

[logo]: https://upload.wikimedia.org/wikipedia/commons/thumb/2/2d/TI-84_Plus_graphing.jpg/151px-TI-84_Plus_graphing.jpg "TI 84"

If you ever had a scientific calculator like the TI-84 I had in school, you might have tried to press the `cos` key repeatedly. It really did not matter what was on screen before; in just a few presses you would always arrive to the same number: `0.73908...` You may not have been aware of this then, but you were actually solving the equation $\cos(x)=x$.

Essentially, given the initial value $x_0$ (whichever value was on the screen at the beginning), you were computing the update:
$$x_{n+1}=\cos(x_n).$$
A Babylonian-flavoured update indeed! We can implement a function which computes the solution as required:

Perhaps not as fast as the Babylonian root method, but the algorithm still finishes in less than 100 iterations!

You can try changing the initial guess over and over and you should see that, once again, it does not really matter what we choose. Positive and negative numbers, large or small, this algorithm always converges to the same solution (the only solution, in fact) in about 90 iterations.

### The Stopping Condition

Note that we have changed the way we check if the iteration has converged. Last week we used the if statement:
```python
if difference == 0:
    return xNext

```
This statement requires that the iteration reaches a fixed point, which only happens when `xNext` equals `x` exactly. We saw this happen in the root algorithm last week but, as we mentioned, that was an iteration with extremely good convergence properties; this will not happen in general.

For our everyday iteration needs, we will have to use a *weaker* condition. Requiring:
```python
if difference < tol:
    return xNext

```
for a small `tol` (tolerance) value is the next best thing! This parameter controls the accuracy of the solution &mdash; we realise that $\|x_{n+1}-x_n\|<\epsilon$ is not exactly the same as $\|x_{n+1}-x_n\|=0$, but the smaller the tolerance $\epsilon$, the smaller the error. Run the cell below to see how the iteration converges to a better value but the number of iterations goes up as `tol` becomes smaller:

In [ ]:
for k in range(1,16):
    cosEquation(10.0**(-k))

### Solving Other Equations

Can all problems in life be solved via iterative algorithms? Unfortunately, no. While this method works well for the cosine equation, it will not work for a general equation
$$f(x)=x.$$

Many things can go wrong. First, a crucially important point: the equation $f(x)=x$ may not have a solution! Even the best of algorithms will fail if we use it to tackle an impossible problem. Did you ever try pressing the `exp` key on your calculator instead? The equation
$$\exp(x)=x$$
does not have real solutions, so the algorithm cannot possibly converge. Try to implement the algorithm; you should see the guess becoming exponentially large and causing an `OverflowError` in just a few iterations.

Second, maybe there is a solution but the convergence is bad! The cosine equation just happened to yield good convergence, but many other equations will not. Try for instance
$$\tan(x)=x.$$

Regardless of how many iterations you set at the beginning, the for loop finishes long before the algorithm has converged. You may also try the sine equation,
$$\sin(x)=x,$$
which is a closely related problem.

Again, bad convergence. It is not that the algorithm is not converging, or that it is converging to the wrong answer &mdash; the algorithm works! It is simply converging very slowly. We could talk about how choosing an initial guess closer to the solution (which is $x=0$, by the way) saves some time in the computation, but altogether this is not a good method to solve equations.

## The name is Newton, Sir Isaac Newton

Since we have seen a **bad** method for solving equations, we should also see a **good** one (a better one, at any rate). We are going to use the *Newton-Raphson method* (sometimes called simply Newton's method), due to Sir Isaac Newton <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Isaac_Newton_signature_ws.svg/320px-Isaac_Newton_signature_ws.svg.png"
     alt=""
     style="display: inline; width: 100px;" />
     and Joseph Raphson <img src="https://upload.wikimedia.org/wikipedia/en/a/ac/Joseph_raphson_signature.jpg"
     alt=""
     style="display: inline; width: 150px;" />.

Say we want to solve a general equation in one variable,
$$f(x)=0.$$
As usual, we make a guess $x_0$ of the true solution $x^*$. Now, if life were easy, $f$ would be a linear function, $f(x)=mx+b$. In this case we know the explicit solution $x^*=-b/m$, but we can instead write the solution as a point which is one step away from the intial guess. Using the gradient of the linear graph and a quick drawing, it is easy to see
$$x^*=x_0-\frac{f(x_0)}{m},$$
or indeed
$$x^*=x_0-\frac{f(x_0)}{f'(x_0)},$$
since $f'(x)=m$ for any $x$.

Of course life is not always easy, and functions are not always linear. However, this method still seems like an intuitive way of moving closer to the solution. The Babylonian within you is probably screaming the iteration $$x_{n+1}=x_n-\frac{f(x_n)}{f'(x_n)}.$$
This iteration is know as the *Newton-Raphson method*.

### Solving a Polynomial
Using Newton's method, solve:
$$x^3-2x^2-11x+12=0.$$
Which solution do you get? How many solutions are there? Can you control which solution you find?

We can reformulate this in a nicer way using a `while` loop. A `while` loop is somehow a mixture of a `for` loop and an `if` statement. Take a look at the example:

In [ ]:
count = 10;

while count > 0:
    count = count-1
    print(count)

Unlike `for` loops, which run for a fixed number of iterations unless they are stopped, `while` loops run as long as the condition is `True`. The loop above counts backwards as long as the count is a positive number, but it stops as soon as it reaches zero.

Such loops can be useful for algorithms such as our `newton` function since we no longer have to choose a maximum number of iterations. However, `while` loops can be dangerous if we are not sure that the condition will ever become `False`. In the example, replacing `count=count-1` by `count=count+1` would create an infinite loop, and the program would never stop running. In general, it can be quite hard to determine whether a program finishes or runs for ever &mdash; this is known as the [Halting Problem](https://en.wikipedia.org/wiki/Halting_problem).